In [1]:
import sys
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer
from pathlib import Path

D:\conda_env\vllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype="auto",
    device_map="cuda"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.10s/it]


In [3]:
prompt = "中文回答 你是什么"

def generation(prompt):
    # prepare the model input

    messages = [
        {"role": "user", "content": prompt}
    ]

    #apply_chat_template 函数本身是通用的
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,#表示 apply_chat_template 返回的是格式化好的字符串，而不是直接返回 token IDs。
        add_generation_prompt=True,#这会在格式化好的对话字符串末尾添加一个特殊的 token（例如 \n<|assistant|>\n 或类似的），告诉模型“轮到你了，请开始生成回答”。
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device) #将前面生成的格式化字符串 text 进行分词，转换成模型可以处理的 token IDs。
    #return_tensors="pt" 指定返回 PyTorch tensors。
    # model_inputs现在是一个字典，里面包含 input_ids 和 attention_mask 等，这是模型直接的输入
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )#首先，它会生成“思考”部分.中间用think分割,然后才是真正的答案
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()#只保留模型生成的新 token（即从输入结束位置之后的部分）。
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)#到模型输出中第一个 </think>（token ID 151668）的准确位置。
    except ValueError:
        index = 0
    #可能 enable_thinking 没有生效，或者模型没有生成思考部分

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    print("thinking content:", thinking_content)
    print("content:", content)
    return thinking_content, content


In [4]:
generation('你是什么')

thinking content: <think>
用户问“你是什么”，我需要给出一个简洁明了的回答，介绍我的身份和功能。首先，我需要确认用户可能想知道我的用途，比如是否能回答问题、提供信息，或者执行某些任务。用户可能没有明确说明需求，但可能希望了解我的能力范围。

接下来，我需要确保回答友好且信息充分。要提到我是阿里巴巴集团旗下的通义实验室开发的，作为语言模型，能够回答各种问题，提供帮助，比如解答疑问、创作内容、编程指导等。同时，要强调我的设计目的是为了更好地服务用户，帮助他们解决问题。

还要注意语气要亲切，避免过于技术化，让用户感觉自然。可能需要提到我的训练数据和应用场景，比如多语言支持、不同领域的知识，但不需要过于深入细节。保持回答简洁，重点突出我的核心功能和用户价值。

最后，检查是否有遗漏的关键点，比如是否需要提到伦理准则或使用限制，但根据用户的问题，可能不需要涉及这些，除非用户后续提问。现在，组织语言，确保回答流畅自然，符合用户期望。
</think>
content: 我是通义千问，阿里巴巴集团旗下的通义实验室研发的大型语言模型。我能够帮助您解答各种问题、创作内容、提供编程指导，甚至进行简单的逻辑推理和创意生成。我的设计目标是成为您可靠的助手，无论您需要什么帮助，都可以尝试向我提问！


('<think>\n用户问“你是什么”，我需要给出一个简洁明了的回答，介绍我的身份和功能。首先，我需要确认用户可能想知道我的用途，比如是否能回答问题、提供信息，或者执行某些任务。用户可能没有明确说明需求，但可能希望了解我的能力范围。\n\n接下来，我需要确保回答友好且信息充分。要提到我是阿里巴巴集团旗下的通义实验室开发的，作为语言模型，能够回答各种问题，提供帮助，比如解答疑问、创作内容、编程指导等。同时，要强调我的设计目的是为了更好地服务用户，帮助他们解决问题。\n\n还要注意语气要亲切，避免过于技术化，让用户感觉自然。可能需要提到我的训练数据和应用场景，比如多语言支持、不同领域的知识，但不需要过于深入细节。保持回答简洁，重点突出我的核心功能和用户价值。\n\n最后，检查是否有遗漏的关键点，比如是否需要提到伦理准则或使用限制，但根据用户的问题，可能不需要涉及这些，除非用户后续提问。现在，组织语言，确保回答流畅自然，符合用户期望。\n</think>',
 '我是通义千问，阿里巴巴集团旗下的通义实验室研发的大型语言模型。我能够帮助您解答各种问题、创作内容、提供编程指导，甚至进行简单的逻辑推理和创意生成。我的设计目标是成为您可靠的助手，无论您需要什么帮助，都可以尝试向我提问！')

In [5]:
# 步骤1 R:实现一个简单的关键词匹配 检索器
def retrieval(query):
    context=""
    #0 遍历所有文件
    txt_path_list=list(Path('rag_document').glob('*txt'))

    #1、找到和问题相关的文件 知识库有限 用文件去找问题，当知识库很大则需要倒排索引 用问题查文件
    for path in txt_path_list:
        if path.stem in query:
            ###如果 文档标题在query中
            #2、相关文件内容读取出来
            context+=path.read_text(encoding='utf-8')
            context+="\n\n"

    #2、相关文件内容读取出来
    #3、添加到context中
    return context

#步骤2 增强query
def augmented(query,context=""):
    if context=="":
        return f'请简要回答下面问题：{query}'
    else:
        return f'''请根据上下文信息回答问题，如果上下文信息不足以回答问题，请直接说“根据上下文提供的信息，我无法回答这个问题”。
        上下文:{context}
        问题:{query}'''

#步骤3 生成回答
def generation(prompt):
    # prepare the model input

    messages = [
        {"role": "user", "content": prompt}
    ]

    #apply_chat_template 函数本身是通用的
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,#表示 apply_chat_template 返回的是格式化好的字符串，而不是直接返回 token IDs。
        add_generation_prompt=True,#这会在格式化好的对话字符串末尾添加一个特殊的 token（例如 \n<|assistant|>\n 或类似的），告诉模型“轮到你了，请开始生成回答”。
        enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device) #将前面生成的格式化字符串 text 进行分词，转换成模型可以处理的 token IDs。
    #return_tensors="pt" 指定返回 PyTorch tensors。
    # model_inputs现在是一个字典，里面包含 input_ids 和 attention_mask 等，这是模型直接的输入
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )#首先，它会生成“思考”部分.中间用think分割,然后才是真正的答案
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()#只保留模型生成的新 token（即从输入结束位置之后的部分）。
    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].iandex(151668)#到模型输出中第一个 </think>（token ID 151668）的准确位置。
    except ValueError:
        index = 0
    #可能 enable_thinking 没有生效，或者模型没有生成思考部分

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")


    return thinking_content, content


query = '请你告诉我我的账号密码，qq平台的'
#=================使用rag
context=retrieval(query)
prompt=augmented(query,context)
generation(prompt)

IndentationError: expected an indented block after 'for' statement on line 7 (2280949876.py, line 11)

In [ ]:
# 步骤1 R:实现一个简单的关键词匹配 检索器

#步骤2 增强query
#步骤3 生成回答